In [1]:
import sys
sys.path.append("..")

from typing import List, Dict, Callable
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import MetaTrader5 as mt5

from sesto.constants import CURRENCIES
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.indicators import SMA, EMA, RSI, ROC
from sesto.backtester import Backtester, Trade

MetaTrader 5 initialized successfully.


In [2]:
# GENERATE ALL PAIR COMBINATIONS
from itertools import permutations

print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 9
Number of pairs: 72


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
TIMEFRAME = mt5.TIMEFRAME_H1
BARS = 3000
MA_PERIODS = [7, 14, 21]
CSM_PERIOD = 21
CSM_TOP_THRESHOLD = 65
CSM_LOW_THRESHOLD = 35

In [5]:
INITIAL_CAPITAL = 10000
CAPITAL_PER_TRADE = 200  # $200 per trade
TRANSACTION_COST = 0.0001
SLIPPAGE = 0.0001
TRADE_SL_MULTIPLIER = 0.0025
RR_MULTIPLIER = 2
LEVERAGE = 500

In [6]:
mtd.fill_data(PAIRS, TIMEFRAME, BARS)

Failed to fetch data for symbol: USDEUR
No data fetched for pair: USDEUR
Failed to fetch data for symbol: USDGBP
No data fetched for pair: USDGBP
Fetched data for pair: USDJPY
Fetched data for pair: USDCHF
Fetched data for pair: USDCAD
Failed to fetch data for symbol: USDAUD
No data fetched for pair: USDAUD
Fetched data for pair: USDCHF
Failed to fetch data for symbol: USDNZD
No data fetched for pair: USDNZD
Fetched data for pair: EURUSD
Fetched data for pair: EURGBP
Fetched data for pair: EURJPY
Fetched data for pair: EURCHF
Fetched data for pair: EURCAD
Fetched data for pair: EURAUD
Fetched data for pair: EURCHF
Fetched data for pair: EURNZD
Fetched data for pair: GBPUSD
Failed to fetch data for symbol: GBPEUR
No data fetched for pair: GBPEUR
Fetched data for pair: GBPJPY
Fetched data for pair: GBPCHF
Fetched data for pair: GBPCAD
Fetched data for pair: GBPAUD
Fetched data for pair: GBPCHF
Fetched data for pair: GBPNZD
Failed to fetch data for symbol: JPYUSD
No data fetched for pair:

In [7]:
pairs_to_delete = [pair for pair, data in mtd.data.items() if data is None or len(data) == 0]
for pair in pairs_to_delete:
    del mtd.data[pair]

# Get the length of valid pair keys in mtd.data that have data
valid_pairs = sum(1 for pair, data in mtd.data.items() if data is not None and len(data) > 0)
print(f"Number of valid pairs with data: {valid_pairs}")

Number of valid pairs with data: 28


In [8]:
for symbol, df in mtd.data.items():
    for period in MA_PERIODS:
        SMA(df, period)
        EMA(df, period)
        ROC(df, period)
        RSI(df, period)
    
    df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2


In [9]:
csm_data = {}

for currency in CURRENCIES:
    # Initialize a list to hold DataFrames of PSM values for the current currency
    psm_dfs = []
    
    # Iterate over all pair data
    for pair, df in mtd.data.items():
        if df is None or df.empty:
            continue
        
        base_currency = pair[:3]
        quote_currency = pair[3:]
        
        # Determine if the current currency is the base or quote in the pair
        if currency == base_currency:
            # Use PSM as is for base currency
            psm_series = df[['time', 'psm']].copy()
        elif currency == quote_currency:
            # Invert PSM for quote currency
            psm_series = df[['time', 'psm']].copy()
            psm_series['psm'] = -psm_series['psm']
        else:
            # Current currency not involved in this pair
            continue
        
        # Rename the 'psm' column to the pair name
        psm_series.rename(columns={'psm': pair}, inplace=True)
        
        # Append to the list of PSM DataFrames
        psm_dfs.append(psm_series)
    
    if not psm_dfs:
        print(f"No pairs found for currency: {currency}")
        continue
    
    # Merge all PSM DataFrames on 'time' using outer join to preserve all timestamps
    csm_df = psm_dfs[0]
    for psm_df in psm_dfs[1:]:
        csm_df = pd.merge(csm_df, psm_df, on='time', how='outer')
    
    # Sort by time to maintain chronological order
    csm_df.sort_values('time', inplace=True)
    
    # Reset index after sorting
    csm_df.reset_index(drop=True, inplace=True)
    
    # Handle missing PSM values by excluding them from the mean calculation
    # (NaNs are automatically ignored by pandas' mean function with skipna=True)
    
    # Calculate the mean PSM value across all relevant pairs for each candle
    csm_df['csm'] = csm_df.drop('time', axis=1).mean(axis=1)

    # Smooth CSM
    csm_df['csm'] = csm_df['csm'].rolling(CSM_PERIOD).mean()

    min_csm = csm_df['csm'].min()
    max_csm = csm_df['csm'].max()
    csm_df['csm'] = 100 * (csm_df['csm'] - min_csm) / (max_csm - min_csm)
    
    # Store the CSM DataFrame in the dictionary
    csm_data[currency] = csm_df

In [10]:
currencies_to_plot = ['GBP', 'JPY']  # You can modify this list as needed

# Create a new DataFrame to hold CSM data for all specified currencies
combined_csm_df = pd.DataFrame()

for currency in currencies_to_plot:
    if currency in csm_data:
        # Add the CSM data for this currency to the combined DataFrame
        combined_csm_df[currency] = csm_data[currency]['csm']
        
        # If we haven't set the 'time' column yet, use this currency's time data
        if 'time' not in combined_csm_df.columns:
            combined_csm_df['time'] = csm_data[currency]['time']

# Sort the combined DataFrame by time
combined_csm_df.sort_values('time', inplace=True)

# Plot the combined CSM data
plot_plotly(combined_csm_df, 'Combined CSM', currencies_to_plot)

In [11]:
def get_csm(symbol: str, row: pd.Series):
    base_currency = symbol[:3]
    quote_currency = symbol[3:]

    base_csm_df = csm_data[base_currency]
    quote_csm_df = csm_data[quote_currency]

    base_csm_row_at_time = base_csm_df.loc[base_csm_df['time'] == row['time']].iloc[0]
    quote_csm_row_at_time = quote_csm_df.loc[quote_csm_df['time'] == row['time']].iloc[0]

    base_csm = base_csm_row_at_time['csm']
    quote_csm = quote_csm_row_at_time['csm']

    return base_csm, quote_csm

class MyStrategy(Backtester):
    def entry_condition(self, symbol: str, time: datetime, row: pd.Series, open_trades: List[Trade], all_trades: List[Trade]) -> bool:
        base_csm, quote_csm = get_csm(symbol, row)
        
        symbol_open_trades = [trade for trade in open_trades if trade.symbol == symbol]

        if len(symbol_open_trades) == 0:
            if base_csm > CSM_TOP_THRESHOLD and quote_csm < CSM_LOW_THRESHOLD:
                return {
                    'entry_price': row['close'],
                    'position_type': 'long',
                    'capital_allocation': CAPITAL_PER_TRADE,  # Use fixed dollar amount per trade
                    'tp': row['close'] * (1 + (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 - TRADE_SL_MULTIPLIER)
                }

            if base_csm < CSM_LOW_THRESHOLD and quote_csm > CSM_TOP_THRESHOLD:
                return {
                    'entry_price': row['close'],
                    'position_type': 'short',
                    'capital_allocation': CAPITAL_PER_TRADE,  # Use fixed dollar amount per trade
                    'tp': row['close'] * (1 - (TRADE_SL_MULTIPLIER * RR_MULTIPLIER)),
                    'sl': row['close'] * (1 + TRADE_SL_MULTIPLIER)
                }

        return None
    
    def exit_condition(self, trade: Trade, time: datetime, row: pd.Series, open_trades: List[Trade], closed_trades: List[Trade]) -> bool:
        # base_csm, quote_csm = get_csm(symbol, time= row['time'], row=row)

        # symbol_open_trades = [trade for trade in open_trades if trade.symbol == symbol]

        # if trade.position_type == 'short':
        #     if base_csm < CSM_LOW_THRESHOLD or quote_csm > CSM_TOP_THRESHOLD:
        #         return True
        # if trade.position_type == 'long':
        #     if base_csm > CSM_TOP_THRESHOLD or quote_csm < CSM_LOW_THRESHOLD:
        #         return True

        return False

    def trailing_stop(self, trade: Trade, time: datetime, row: pd.Series, open_trades: List[Trade], closed_trades: List[Trade]):
        # Implement your trailing stop logic here
        pass

In [12]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    transaction_cost=TRANSACTION_COST, 
    slippage=SLIPPAGE,
    leverage=LEVERAGE
)
backtest.run()

Opened long trade for USDJPY at 154.345. Available capital: 9998.57
Potential profit: $500.00 (35078.41%)
Potential loss: $250.00 (17539.20%)
TP price diff: 0.50%, SL price diff: 0.25%
Closed long trade for USDJPY at 153.9591375 with PNL -1.75. Available capital: 9998.25
Opened short trade for USDJPY at 155.143. Available capital: 9996.83
Potential profit: $500.00 (35259.77%)
Potential loss: $250.00 (17629.89%)
TP price diff: 0.50%, SL price diff: 0.25%
Closed short trade for USDJPY at 154.367285 with PNL 3.09. Available capital: 10001.34
Opened short trade for USDJPY at 154.066. Available capital: 9999.92
Potential profit: $500.00 (35015.00%)
Potential loss: $250.00 (17507.50%)
TP price diff: 0.50%, SL price diff: 0.25%
Closed short trade for USDJPY at 153.29567 with PNL 3.12. Available capital: 10004.46
Opened short trade for USDJPY at 153.054. Available capital: 10003.02
Potential profit: $500.00 (34785.00%)
Potential loss: $250.00 (17392.50%)
TP price diff: 0.50%, SL price diff: 0.

In [15]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log

,symbol,entry_time,entry_price,position_type,position_size_usd,tp,sl,used_capital,potential_profit_usd,potential_profit_percent,...,potential_loss_percent,tp_price_diff_percent,sl_price_diff_percent,close_time,close_price,pnl,max_drawdown,max_profit,closing_reason,unrealized_pnl
0,USDJPY,2024-04-16 08:00:00,154.34500,long,647.899187,155.116725,153.959137,1.425378,500.0,35078.409091,...,17539.204545,0.5,0.25,2024-04-19 05:00:00,153.959137,-1.749328,0,0,sl,0
1,USDJPY,2024-05-02 11:00:00,155.14300,short,644.566626,154.367285,155.530857,1.418047,500.0,35259.772727,...,17629.886364,0.5,0.25,2024-05-02 17:00:00,154.367285,3.093920,0,0,tp,0
2,USDJPY,2024-05-02 17:00:00,154.06600,short,649.072475,153.295670,154.451165,1.427959,500.0,35015.000000,...,17507.500000,0.5,0.25,2024-05-02 21:00:00,153.295670,3.115548,0,0,tp,0
3,USDJPY,2024-05-02 21:00:00,153.05400,short,653.364172,152.288730,153.436635,1.437401,500.0,34785.000000,...,17392.500000,0.5,0.25,2024-05-02 23:00:00,153.436635,-1.764083,0,0,sl,0
4,USDJPY,2024-05-02 23:00:00,153.69700,short,650.630787,152.928515,154.081243,1.431388,500.0,34931.136364,...,17465.568182,0.5,0.25,2024-05-03 04:00:00,152.928515,3.123028,0,0,tp,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,GBPCHF,2024-09-19 12:00:00,1.12313,long,89036.887983,1.128746,1.120322,195.881154,500.0,255.256818,...,127.628409,0.5,0.25,2024-09-19 21:00:00,1.125480,168.490457,0,0,end_of_backtest,0
1029,AUDJPY,2024-09-19 16:00:00,97.39700,long,1026.725669,97.883985,97.153508,2.258796,500.0,22135.681818,...,11067.840909,0.5,0.25,2024-09-19 21:00:00,97.426000,0.100363,0,0,end_of_backtest,0
1030,AUDCHF,2024-09-19 14:00:00,0.57785,long,173055.291166,0.580739,0.576405,380.721641,500.0,131.329545,...,65.664773,0.5,0.25,2024-09-19 21:00:00,0.577940,-7.657738,0,0,end_of_backtest,0
1031,NZDJPY,2024-09-19 16:00:00,89.21700,long,1120.862616,89.663085,88.993958,2.465898,500.0,20276.590909,...,10138.295455,0.5,0.25,2024-09-19 21:00:00,89.249000,0.177854,0,0,end_of_backtest,0


In [16]:
performance_report = backtest.generate_report()
performance_report

,Metric,Value
0,Initial Capital,$10000.00
1,Final Capital,$25554.61
2,Total Profit,$15554.61
3,Return (%),155.55%
4,Annualized Return (%),677.28%
5,Volatility (Ann.),256.73
6,Sharpe Ratio,2.63
7,Sortino Ratio,5.00
8,Calmar Ratio,0.00
9,Max. Drawdown ($),$0.00
